# Objective 

Email Parser can be used to create database of emails in a csv file

## Imports

In [78]:
import imapclient
import pyzmail
import html2text
import re

## Input

In [80]:
username = input('Enter the email id    : ')
password = input('Enter the password    : ')
folder   = input('Enter the folder name : ')

Enter the email id    : products@sunarctechnologies.com
Enter the password    : sunarc@123
Enter the folder name : INBOX


## Logging in to the IMAP Server

In [81]:
imapObj = imapclient.IMAPClient('imap.gmail.com', ssl=True)
imapObj.login(username, password)

b'products@sunarctechnologies.com authenticated (Success)'

In [82]:
# Selecting the folder
imapObj.select_folder(folder, readonly=True)

{b'EXISTS': 117,
 b'FLAGS': (b'\\Answered',
  b'\\Flagged',
  b'\\Draft',
  b'\\Deleted',
  b'\\Seen',
  b'$NotPhishing',
  b'$Phishing'),
 b'HIGHESTMODSEQ': 21266,
 b'PERMANENTFLAGS': (),
 b'READ-ONLY': [b''],
 b'RECENT': 0,
 b'UIDNEXT': 128,
 b'UIDVALIDITY': 1}

In [127]:
UIDs = imapObj.gmail_search('Question') # Returns UIDs of every message in the currently selected folder
print(UIDs)

[7, 33, 39, 41, 43, 44, 76, 78, 80, 82, 85, 100, 123, 125]


## Fetching Emails  

In [128]:
rawMessages = imapObj.fetch(UIDs, ['BODY[]'])
print("The total no. of mail : ",len(rawMessages))

The total no. of mail :  14


In [149]:
message = pyzmail.PyzMessage.factory(rawMessages[82][b'BODY[]'])

In [150]:
message.get_subject()

'Fwd: Fwd: New Question Posted at Magento Marketplace'

In [151]:
message.get_addresses('to')

[('products@sunarctechnologies.com', 'products@sunarctechnologies.com')]

In [152]:
message.get_addresses('cc')

[]

In [153]:
def parser(message):
    if message.text_part is not None and message.html_part is not None:
        multipart = True
    else:
        multipart = False

    if message.text_part is not None:
        try:
            body = message.text_part.get_payload().decode(message.text_part.charset)
        except TypeError:
            body = message.text_part.get_payload()

    if message.html_part is not None and multipart is False:
        try:
            body = html2text.html2text(message.html_part.get_payload().decode(message.html_part.charset))
        except Exception:
            raise Systemexit
    return body

In [154]:
def cleanhtml(sentence):
    """ Cleans the word of any html-tag.
    
    Args:
      sentence: Takes Sentence
      
    Returns:
      Sentence without any html tags in it.
    """
    cleanr = re.compile('<.*?')
    cleantext = re.sub(cleanr, ' ', sentence)
    
    return cleantext

In [155]:
def cleanpunc(sentence):
    """ Cleans the punctuation in the sentence.
    
    Args:
      sentence: A sentence
      
    Returns:
      A sentence after removing punctuations. 
    """
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    
    return  cleaned

In [156]:
body = parser(message)
body = cleanhtml(body)
body = cleanpunc(body)
print(body)

----Forwarded using *Multi-Forward Chrome Extension*
 https:  chrome google com webstore detail multi-forward-for-gmail jjmdplljmniahpamcmabdnahmjdlikpm>
---
From: Rajneesh Vyas
Date: Wed Feb 21 2018 22:02:44 GMT+1100  AEDT 
Subject: Fwd: New Question Posted at Magento Marketplace
To: vasim ahmad   Monika Agarawal


Rajneesh Vyas 
SunArc Technologies Pvt  Ltd 
29 30 Panchshati Circle 
Bikaner  Rajasthan  India
Phone: +91 151 2200810
Cell Phone: +91 94 14140013
Email: rajneesh vyas@sunarctechnologies com
www sunarctechnologies com

 https:  www facebook com sunarctechnologies>
 https:  www linkedin com company sunarc-technologies>
 https:  twitter com SunArc_tech>
 https:  www youtube com user sunarcstudios>



*Latest Releases :*

Opportunity - A short movie by IT Professionals
 http:  sunarcstudios com movies opportunity >

Boxing Ball - iPhone iPad Game  http:  sunarcstudios com games boxing-ball >

Toons Balloons - iPhone iPad Game
 http:  sunarcstudios com games toons-balloons >

P